In [2]:
#!pip install imblearn

<a href="https://colab.research.google.com/github/helene-78/Accidents/blob/main/prep_df_utiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE # Pour avoir 50 % de 0 et 50 % de 1 dans la colonne presence_accident
from sklearn.linear_model import LogisticRegression # Regression logistique
from sklearn import metrics
from sklearn.model_selection import RepeatedKFold #KFold répété, pour 
from sklearn.model_selection import train_test_split

**Fusion des différents dataframes**

In [9]:
df_accidents = pd.read_csv('C://Users/srond/Documents/GitHub/Accidents/caracteristiques-2019.csv', sep=";")
df_usagers = pd.read_csv('C://Users/srond/Documents/GitHub/Accidents/usagers-2019.csv', sep=";")
df_lieux = pd.read_csv('C://Users/srond/Documents/GitHub/Accidents/lieux-2019.csv', sep=";")
df_vehicules = pd.read_csv('C://Users/srond/Documents/GitHub/Accidents/vehicules-2019.csv', sep=";")
df_test = df_accidents.merge(df_usagers.merge(df_lieux.merge(df_vehicules))) #dataframe à traiter

In [11]:
df_test.head()

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,...,situ,vma,senc,catv,obs,obsm,choc,manv,motor,occutc
0,201900000001,30,11,2019,01:30,4,93,93053,1,1,...,1,70,2,7,0,2,5,23,1,NaN
1,201900000001,30,11,2019,01:30,4,93,93053,1,1,...,1,70,2,7,0,2,5,23,1,NaN
2,201900000001,30,11,2019,01:30,4,93,93053,1,1,...,1,70,2,17,1,0,3,11,1,NaN
3,201900000002,30,11,2019,02:50,3,93,93066,1,1,...,1,70,1,7,4,0,1,0,1,NaN
4,201900000003,28,11,2019,15:15,1,92,92036,1,1,...,1,90,1,7,0,2,1,2,1,NaN


**Pour supprimer les colonnes inutiles à l'analyse de données, supprimer les NaN et avoir une colonne heure au bon format**

In [15]:
df_utiles = df_test.drop(axis=1, labels=['Num_Acc', 'com', 'adr', 'v1', 'id_vehicule', 'num_veh', 'lartpc', 'larrout', 'occutc', 'v2', 'voie'])
df_utiles = df_utiles.dropna()
df_utiles["hrmn"] = pd.to_datetime(df_utiles["hrmn"], format='%H:%S')
df_utiles.head()
df_utiles.dtypes

jour                int64
mois                int64
an                  int64
hrmn       datetime64[ns]
lum                 int64
dep                object
agg                 int64
int                 int64
atm                 int64
col                 int64
lat                object
long               object
place               int64
catu                int64
grav                int64
sexe                int64
an_nais             int64
trajet              int64
secu1               int64
secu2               int64
secu3               int64
locp                int64
actp               object
etatp               int64
catr                int64
circ                int64
nbv                 int64
vosp                int64
prof                int64
pr                 object
pr1                object
plan                int64
surf                int64
infra               int64
situ                int64
vma                 int64
senc                int64
catv                int64
obs         

In [16]:
for i in df_utiles.index:
    df_utiles["hrmn"][i] = (int(df_utiles["hrmn"][i].hour))*60 + (int(df_utiles["hrmn"][i].minute))


<ipython-input-16-321a483515aa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_utiles["hrmn"][i] = (int(df_utiles["hrmn"][i].hour))*60 + (int(df_utiles["hrmn"][i].minute))
D:\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


**Création et remplissage d'une colonne "presence_deces" qui vaut 1 en cas d'accident mortel, 0 sinon**

In [17]:
df_utiles["hrmn"] = df_utiles["hrmn"].astype('int64')
df_utiles["dep"] = df_utiles["dep"].replace(["2B"], "999") #On remplace les codes des départements de la Corse par des int
df_utiles["dep"] = df_utiles["dep"].replace(["2A"], "998")
df_utiles["actp"] = df_utiles["actp"].replace(["A"], "14") #On remplace les actions du piéton non encodées par des chiffres
df_utiles["actp"] = df_utiles["actp"].replace(["B"], "15")
df_utiles["dep"] = df_utiles["dep"].astype('int64')
df_utiles["presence_deces"] = 0
for i in df_utiles.index:
    if df_utiles["grav"][i] == 2:
        df_utiles["presence_deces"][i] = 1
df_utiles.to_csv('df_utiles.csv')

<ipython-input-17-0d06889cd100>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_utiles["presence_deces"][i] = 1
